In [6]:
from typing import List, Tuple, Optional
import math

# -----------------------------
# Bendros pagalbinės
# -----------------------------
def print_matrix(M: List[List[float]], title: str = "", fmt: str = "{:11.6g}"):
    if title:
        print(f"\n{title}\n" + "-" * len(title))
    for row in M:
        print(" ".join(fmt.format(v) for v in row))
    print()

def print_vector(v: List[float], title: str = "", fmt: str = "{:11.6g}"):
    if title:
        print(f"\n{title}\n" + "-" * len(title))
    print(" ".join(fmt.format(x) for x in v))
    print()

def deep_copy_matrix(A: List[List[float]]) -> List[List[float]]:
    return [row[:] for row in A]

def eye(n: int) -> List[List[float]]:
    return [[1.0 if i==j else 0.0 for j in range(n)] for i in range(n)]

def matmul(A: List[List[float]], B: List[List[float]]) -> List[List[float]]:
    n, m, p = len(A), len(A[0]), len(B[0])
    out = [[0.0]*p for _ in range(n)]
    for i in range(n):
        for k in range(m):
            aik = A[i][k]
            if aik == 0.0: continue
            for j in range(p):
                out[i][j] += aik * B[k][j]
    return out

def matvec(A: List[List[float]], x: List[float]) -> List[float]:
    return [sum(A[i][j]*x[j] for j in range(len(x))) for i in range(len(A))]

def vec_sub(a: List[float], b: List[float]) -> List[float]:
    return [a[i]-b[i] for i in range(len(a))]

def vec_add(a: List[float], b: List[float]) -> List[float]:
    return [a[i]+b[i] for i in range(len(a))]

def norm2_vec(v: List[float]) -> float:
    return math.sqrt(sum(x*x for x in v))

def transpose(A: List[List[float]]) -> List[List[float]]:
    return [list(row) for row in zip(*A)]

def copy_col_block(A: List[List[float]], r0: int, c0: int) -> List[List[float]]:
    """Return submatrix A[r0:, c0:] (deep copy)."""
    return [row[c0:] for row in A[r0:]]

def set_block(A: List[List[float]], r0: int, c0: int, B: List[List[float]]):
    """Set A[r0:, c0:] = B (in place)."""
    for i in range(len(B)):
        for j in range(len(B[0])):
            A[r0+i][c0+j] = B[i][j]

def augment(A: List[List[float]], b: List[float]) -> List[List[float]]:
    return [row + [b[i]] for i, row in enumerate(A)]

def check_solution(A: List[List[float]], x: Optional[List[float]], b: List[float], tag: str):
    if x is None:
        print(f"[{tag}] sprendinio nėra ką tikrinti.")
        return
    Ax = matvec(A, x)
    r  = vec_sub(b, Ax)
    print_vector(Ax, f"{tag}: A*x")
    print_vector(r,  f"{tag}: liekana r = b - A*x")
    print(f"{tag}: ||x||2 = {norm2_vec(x):.12g},  ||r||2 = {norm2_vec(r):.12g}\n")

# -----------------------------
# 1) ATSPINDŽIO (HOUSEHOLDER) VEIKSMAS VEKTORIUI
# -----------------------------
def householder_for_vector(z: List[float], verbose: bool=True) -> Tuple[List[List[float]], List[float]]:
    """
    Pagal skaidres: Q = I - 2 ω ω^T, ω = (z - zp)/||z - zp||,
    kur zp = [sign(z0)*||z||, 0, ..., 0]^T.
    Gražina (Q, z_reflected=Q z).
    """
    n = len(z)
    if verbose:
        print_vector(z, "Atspindis: pradinė z")

    nz = norm2_vec(z)
    if nz < 1e-15:
        if verbose:
            print("Atspindis: z ≈ 0 → Q = I (žingsnis praleidžiamas).")
        return (eye(n), z[:])

    zp = [0.0]*n
    zp[0] = (1.0 if z[0] >= 0 else -1.0)*nz
    omega = [z[i]-zp[i] for i in range(n)]
    nom = norm2_vec(omega)
    if nom < 1e-15:
        if verbose:
            print("Atspindis: z jau sutampa su zp → Q = I.")
        return (eye(n), z[:])

    omega = [w/nom for w in omega]

    # Q = I - 2 ω ω^T
    Q = eye(n)
    for i in range(n):
        for j in range(n):
            Q[i][j] -= 2.0*omega[i]*omega[j]

    z_ref = matvec(Q, z)

    if verbose:
        print_vector(zp,    "Atspindis: z' (tikslo vektorius)")
        print_vector(omega, "Atspindis: omega (normalė)")
        print_matrix(Q,     "Atspindis: Q = I - 2*omega*omega^T")
        print_vector(z_ref, "Atspindis: Q*z (pakeistas vektorius)")
    return Q, z_ref

# -----------------------------
# 2) ATSPINDŽIO ALGORITMAS spręsti Ax=b (trianguliacija + back-sub)
# -----------------------------
def solve_by_reflection(A_in: List[List[float]], b_in: List[float], verbose: bool=True) -> Optional[List[float]]:
    A = deep_copy_matrix(A_in)
    b = b_in[:]
    n = len(A)

    if verbose:
        print_matrix(augment(A,b), "Atspindžio metodas: pradinė [A|b]")

    # Tiesioginis etapas (stulpeliais, subblokais i:n, i:n)
    for i in range(n-1):
        # darbo stulpelis z = A[i:n, i]
        z = [A[r][i] for r in range(i, n)]
        # sudarom Q_i mažinantį subbloką
        Qi, _ = householder_for_vector(z, verbose=verbose)
        # pritaikome [Qi 0; 0 I] kairėje A ir b
        sub = copy_col_block(A, i, i)               # A[i:, i:]
        sub2 = matmul(Qi, sub)                      # Qi * sub
        set_block(A, i, i, sub2)

        # ir b segmentui
        bi = b[i:]
        bi2 = matvec(Qi, bi)
        for r in range(i, n):
            b[r] = bi2[r-i]

        if verbose:
            print_matrix(augment(A,b), f"Po atspindžio žingsnio i={i} (A|b)")

    # Atgalinė eiga (gali būti singuliarus atvejis)
    x = [0.0]*n
    for i in range(n-1, -1, -1):
        # jei įstrižainė ~0, sprendinys neapibrėžtas/nesuderinamas — pranešam
        if abs(A[i][i]) < 1e-15:
            print(f"Atspindžio metodas: A[{i},{i}]≈0 → sistema gali būti singuliari. Šį kintamąjį paliekame 0.")
            # dvasingiau būtų laisvieji kintamieji — čia pateikiam bazinį sprendinį
            continue
        s = sum(A[i][j]*x[j] for j in range(i+1, n))
        x[i] = (b[i] - s)/A[i][i]
        if verbose:
            print(f"  BackSub: x[{i}] = (b[{i}] - suma)/A[{i},{i}] = {(b[i]-s):.6g}/{A[i][i]:.6g} = {x[i]:.12g}")

    print_vector(x, "Atspindžio metodas: sprendinys x")
    check_solution(A_in, x, b_in, "Atspindžio metodas")
    return x

# -----------------------------
# 3) PATIKRINIMAI: ortogonalumas ir simetrija
# -----------------------------
def is_orthogonal(Q: List[List[float]], tol: float=1e-10, verbose: bool=True) -> bool:
    QTQ = matmul(transpose(Q), Q)
    n = len(Q)
    I = eye(n)
    diff = [[QTQ[i][j]-I[i][j] for j in range(n)] for i in range(n)]
    if verbose:
        print_matrix(Q,   "Ortogonalumo patikra: Q")
        print_matrix(QTQ, "Ortogonalumo patikra: Q^T Q")
        print_matrix(diff,"Ortogonalumo patikra: Q^T Q - I")
    max_abs = max(abs(diff[i][j]) for i in range(n) for j in range(n))
    ok = max_abs <= tol
    print(f"Ortogonalumo patikra: max|Q^TQ - I| = {max_abs:.3e} → {'TAIP' if ok else 'NE'} (tol={tol})")
    return ok

def is_symmetric(A: List[List[float]], tol: float=1e-10, verbose: bool=True) -> bool:
    AT = transpose(A)
    n, m = len(A), len(A[0])
    diff = [[A[i][j]-AT[i][j] for j in range(m)] for i in range(n)]
    if verbose:
        print_matrix(A,  "Simetrijos patikra: A")
        print_matrix(AT, "Simetrijos patikra: A^T")
        print_matrix(diff,"Simetrijos patikra: A - A^T")
    max_abs = max(abs(diff[i][j]) for i in range(n) for j in range(m))
    ok = max_abs <= tol
    print(f"Simetrijos patikra: max|A - A^T| = {max_abs:.3e} → {'TAIP' if ok else 'NE'} (tol={tol})")
    return ok

# -----------------------------
# 4) QR SKAIDA per atspindžius (Householder), + sprendimas
# -----------------------------
def qr_decomposition(A_in: List[List[float]], verbose: bool=True) -> Tuple[List[List[float]], List[List[float]]]:
    """
    Atitinka skaidrių konstrukciją: A -> R, akumuliuojant Q (Q[:, i:] = Q[:, i:] @ Qi).
    """
    A = deep_copy_matrix(A_in)
    n = len(A)
    Q = eye(n)

    if verbose:
        print_matrix(A, "QR skaida: pradinė A")

    for i in range(n-1):
        # subvektorius z = A[i:, i]
        z = [A[r][i] for r in range(i, n)]
        Qi_small, _ = householder_for_vector(z, verbose=verbose)

        # pritaikom A[i:, i:] = Qi_small @ A[i:, i:]
        sub = copy_col_block(A, i, i)
        sub2 = matmul(Qi_small, sub)
        set_block(A, i, i, sub2)

        # Q[:, i:] = Q[:, i:] @ Qi_small
        # sudarome blokinę matricą \tilde{Qi}
        Qi_full = eye(n)
        for r in range(n-i):
            for c in range(n-i):
                Qi_full[i+r][i+c] = Qi_small[r][c]
        Q = matmul(Q, Qi_full)

        if verbose:
            print_matrix(A, "QR: tarpinė A (->R) po žingsnio")
            print_matrix(Q, "QR: sukaupta Q po žingsnio")

    R = A
    if verbose:
        print_matrix(Q, "QR: galutinė Q")
        print_matrix(R, "QR: galutinė R (viršutinė trikampė)")

    return Q, R

def back_substitution(R: List[List[float]], y: List[float], verbose: bool=True) -> List[float]:
    n = len(R)
    x = [0.0]*n
    if verbose:
        print_matrix(R, "Atgalinė eiga (QR): R")
        print_vector(y, "Atgalinė eiga (QR): y = Q^T b")
    for i in range(n-1, -1, -1):
        if abs(R[i][i]) < 1e-15:
            print(f"QR: R[{i},{i}]≈0 → singuliarus/įsilaidės atvejis (laisvas kintamasis=0).")
            continue
        s = sum(R[i][j]*x[j] for j in range(i+1, n))
        x[i] = (y[i] - s)/R[i][i]
        if verbose:
            print(f"  x[{i}] = {(y[i]-s):.6g}/{R[i][i]:.6g} = {x[i]:.12g}")
    return x

def qr_solve(A_in: List[List[float]], b: List[float], verbose: bool=True) -> Optional[List[float]]:
    Q, R = qr_decomposition(A_in, verbose=verbose)
    # y = Q^T b
    QT = transpose(Q)
    y = matvec(QT, b)
    print_vector(y, "QR: y = Q^T b")
    x = back_substitution(R, y, verbose=verbose)
    print_vector(x, "QR: sprendinys x")
    check_solution(A_in, x, b, "QR")
    # patikrinimai
    is_orthogonal(Q, verbose=verbose)
    return x

# ============================================================
# MAIN – užkomentuoti paleidimai
# ============================================================
def main():
    # Pavyzdiniai duomenys
    A = [
        [1.0,  1.0,  1.0,  2.0],
        [1.0, -1.0, -1.0,  0.0],
        [2.0,  1.0, -1.0,  2.0],
        [3.0,  1.0,  2.0,  1.0],
    ]
    b = [2.0, 0.0, 9.0, 7.0]

    # --- 1) Vieno vektoriaus atspindys (demo) ---
    # z = [row[0] for row in A]                  # imame 1-ą stulpelį
    # Qz, z_ref = householder_for_vector(z, verbose=True)

    # --- 2) Atspindžio algoritmas (sprendimas Ax=b) ---
    # x_reflect = solve_by_reflection(A, b, verbose=True)

    # --- 3) Patikros: ortogonalioji ir simetrinė matrica ---
    # Q_demo, R_demo = qr_decomposition(A, verbose=True)
    # is_orthogonal(Q_demo, verbose=True)
    # is_symmetric(A, verbose=True)

    # --- 4) QR skaida + sprendimas ---
    x_qr = qr_solve(A, b, verbose=True)

    pass

main()


QR skaida: pradinė A
--------------------
          1           1           1           2
          1          -1          -1           0
          2           1          -1           2
          3           1           2           1


Atspindis: pradinė z
--------------------
          1           1           2           3


Atspindis: z' (tikslo vektorius)
--------------------------------
    3.87298           0           0           0


Atspindis: omega (normalė)
--------------------------
  -0.609016     0.21198    0.423961    0.635941


Atspindis: Q = I - 2*omega*omega^T
----------------------------------
   0.258199    0.258199    0.516398    0.774597
   0.258199    0.910129   -0.179743   -0.269614
   0.516398   -0.179743    0.640515   -0.539228
   0.774597   -0.269614   -0.539228    0.191158


Atspindis: Q*z (pakeistas vektorius)
------------------------------------
    3.87298 5.55112e-17 1.66533e-16 -2.77556e-16


QR: tarpinė A (->R) po žingsnio
------------------------------